#Data Preprocessing

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

# Step 1: Load the Dataset
Jobs_data = pd.read_csv("all_upwork_jobs_next.csv")

# Step 2: Inspect the Dataset
print(Jobs_data.head())
print(Jobs_data.info())


   Unnamed: 0                                              title  \
0           0  experienced media buyer for solar pannel and r...   
1           1                               full stack developer   
2           2                                    smma bubble app   
3           3             talent hunter specialized in marketing   
4           4                                      data engineer   

                                                link  \
0  https://www.upwork.com/jobs/Experienced-Media-...   
1  https://www.upwork.com/jobs/Full-Stack-Develop...   
2  https://www.upwork.com/jobs/SMMA-Bubble-App_%7...   
3  https://www.upwork.com/jobs/Talent-Hunter-Spec...   
4  https://www.upwork.com/jobs/Data-Engineer_%7E0...   

              published_date  is_hourly  hourly_low  hourly_high       budget  \
0  2024-02-17 09:09:54+00:00      False   17.339644    41.970610   500.000000   
1  2024-02-17 09:09:17+00:00      False   17.339644    41.970610  1100.000000   
2  2024-02-

In [ ]:
print(Jobs_data.columns)

Index(['Unnamed: 0', 'title', 'link', 'published_date', 'is_hourly',
       'hourly_low', 'hourly_high', 'budget', 'country', 'avg_salary',
       'salary', 'average_hourly', 'category', 'year_month', 'month', 'year',
       'month_year'],
      dtype='object')


In [ ]:
# Select feature and target columns
X = Jobs_data['budget']  # Replace with the actual column name for text data
y = Jobs_data['avg_salary']  # Replace with the actual column name for the target

# Convert the 'title' column to string type to avoid errors with TfidfVectorizer
X = X.astype(str)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Define a pipeline with TfidfVectorizer and RandomForestRegressor
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Convert text to numerical features
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))  # Regression model
])


In [ ]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Save the trained model for later use
joblib.dump(pipeline, 'job_prediction_model.pkl')
print("Model saved as 'job_prediction_model.pkl'")


Model saved as 'job_prediction_model.pkl'


In [ ]:
# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate and display performance metrics
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.22620425784464843


In [ ]:
# Load the trained model
loaded_model = joblib.load('job_prediction_model.pkl')

# Predict for new data
new_jobs = ["Looking for a Python developer with experience in Flask and APIs."]
predictions = loaded_model.predict(new_jobs)
print("Predictions for new jobs:", predictions)


Predictions for new jobs: [29.06996535]


In [ ]:
import joblib

# Save the model using the correct variable name (pipeline)
joblib.dump(pipeline, 'budget_predictor_model.pkl')

['budget_predictor_model.pkl']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Assuming 'Jobs_data' is your DataFrame
# ... (your code to load Jobs_data) ...

# Define features and target
X = Jobs_data.drop(columns=['title', 'link', 'published_date', 'budget'])
y = Jobs_data['budget']

# Identify categorical features (columns with string values)
categorical_features = X.select_dtypes(include=['object']).columns

# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # sparse=False for compatibility with RandomForestRegressor

# Fit the encoder to the categorical features and transform them
encoded_features = encoder.fit_transform(X[categorical_features])

# Create a DataFrame from the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Drop the original categorical features and concatenate the encoded features
X = X.drop(columns=categorical_features)
X = pd.concat([X, encoded_df], axis=1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.005648565841604368
